# Part 3. Extracting signal from a region of interest

In Part 1, we preprocessed the data (or, at least, explored *how* you could go about preprocessing your data). Let's just use the `fmriprep`-preprocessed data for the remainder of the tutorial. Here, we will extract signal from a region of interest: striatum. Striatum is part of the Basal Ganglia, and has repeatedly been implicated in response caution adjustments (e.g., [Forstmann et al 2008](http://www.pnas.org/content/105/45/17538/#F2); [Van Maanen et al 2011](http://www.jneurosci.org/content/31/48/17488)). In Part 3 and 4 of this tutorial, we'll see if we can replicate this result.

In order to extract signal from a region of interest, we need a mask that tells us where this ROI lies in the brain. Since `fmriprep` registered ('normalized') our data to MNI152-space, we can use a mask from an atlas in MNI152-space.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from nipype.interfaces import fsl
import nilearn.datasets

%matplotlib inline

## A look at masks

In the directory `/data/masks/` you can find various masks, including one for the left and one for the right striatum

In [ ]:
!ls /data/masks/MNI152-2mm

Let's load a mask and see what it looks like.

In [ ]:
str_l = nib.load('/data/masks/MNI152-2mm/STR_L.nii.gz').get_data()
plt.imshow(str_l[:, 68, :].T, origin='lower', cmap=plt.cm.hot)

Note that this is a *probabilistic* map. The colors in the above plot indicate the *probability* for each voxel that is is part of any given individual's left striatum. Further, probabilities of 0 lead to a black pixel in the plot above.

Let's mask all voxels that have a probability of 50% or less of belonging to any of the striatal parts. This way, they won't end up as black pixels when we plot them, but become transparant

In [ ]:
str_l = np.ma.masked_less(str_l, 0.5)

Now, `str_l` is a 3D-matrix indicating for each voxel the probability that it belongs to the left striatum. What does this look like?

In [ ]:
plt.imshow(str_l[:, 68, :].T, origin='lower', cmap=plt.cm.hot)

The background is now white instead of black, which doesn't help much. But now we can superimpose this mask on an MNI152 template

In [ ]:
# Load the MNI template
mni = nib.load('/data/mni_icbm152_nlin_asym_09c/2mm_T1.nii.gz').get_data()

In [ ]:
plt.figure(figsize=(20, 10))

plt.subplot(131)
plt.imshow(mni[:, 68, :].T,origin='lower', cmap=plt.cm.gray)
plt.imshow(str_l[:, 68, :].T,origin='lower', cmap=plt.cm.hot)
plt.grid('off')

plt.subplot(132)
plt.imshow(mni[38, :, :].T,origin='lower', cmap=plt.cm.gray)
plt.imshow(str_l[38, :, :].T,origin='lower', cmap=plt.cm.hot)
plt.grid('off')

plt.subplot(133)
plt.imshow(mni[:, :, 40].T,origin='lower', cmap=plt.cm.gray)
plt.imshow(str_l[:, :, 40].T,origin='lower', cmap=plt.cm.hot)
plt.grid('off')

And that's where left striatum is located probabilistically

Note that the `str_l` variable is a kind of image. You can apply image processing tools to it; for example, to find the center of mass (this is how the slices plotting above were chosen)

In [ ]:
import scipy as sp
from scipy import ndimage
com = ndimage.center_of_mass(str_l)
com

1) Can you make the same plots for right striatum? Find the center of mass, and plot it on an MNI152-brain

In [ ]:
# your code here

## Extracting signal from a masked region
Back to Nipype. Nipype contains interfaces that allows you to extract a time series *meaned* over a region of interest. The interface is called `ImageMeants()`. Let's first see how this works

In [ ]:
# We'll need these libraries below
from nipype.interfaces import fsl
import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio

In [ ]:
!ls /data/deriv/fmriprep/sub-483/func

In [ ]:
extracter = fsl.ImageMeants()
extracter.inputs.in_file = '/data/deriv/fmriprep/sub-483/func/sub-483_task-SAT_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
extracter.inputs.mask = '/data/masks/MNI152-epi/STR_L.nii.gz'

In [ ]:
r = extracter.run()
r.outputs

Now, using `np.loadtxt(<path_to_txt_file>)`, you can load the output. 

2) Can you plot the data?

## Making a pipeline
Now we semi-manually extracted a single subject's time series signal meaned over striatum. We could do this in the same way for all subjects, but it's much more efficient to make a pipeline. Let's make one that extracts the time series meaned over striatum for all participants. First, set-up the nodes we want to use

In [ ]:
# This is the node that extracts signal
extracter = pe.MapNode(fsl.ImageMeants(), iterfield='in_file', name='extract_mean_ts')

Note that this is a [MapNode](https://pythonhosted.org/nipype/api/generated/nipype.pipeline.engine.nodes.html#mapnode) rather than a regular Node; i.e., a Node the receives a list of inputs and applies the same interface to each element of the list.

3) In the cell below, create/fill: 
- the template for where to find the masks
- the template for where to find the functional data (similar to Part 1)
- all subject ids at selector.iterator. 
- fill in the mask names at selector.iterator

In [ ]:
# Here, we set-up a SelectFile-Node
templates = {
    'mask': '/data/masks/MNI152-epi/ ... ',    # NB: folder "MNI152-epi" here is correct - dont use "MNI152-2mm"
    'func': '/data/deriv/fmriprep/ ... '
}

selector = pe.Node(nio.SelectFiles(templates=templates), name='file_selector')
selector.iterables = [('subject_id', [ ... ]),   # enter subejct ids
                      ('mask', [ ... ])] # enter mask names


# A datasink is useful for exporting. Remember the base_directory - this is where the data will be saved later
ds = pe.Node(nio.DataSink(base_directory='/data/extracted_signals'), 
             name='datasink')

Then, combine everything in a Workflow.

4) In the cell below, make the right connections in the workflow: you want to connect the `selector` to the `extractor` in two ways. Remember that you can call `fsl.ImageMeants.help()` to see the input argument names

In [ ]:
workflow = pe.Workflow(base_dir='/data/workflow_folders/', 
                       name='extract_striatum_signal')

# Connect the Nodes
workflow.connect(selector, 'mask', extracter, ...) # complete the connection
workflow.connect(selector, 'func', extracter, ...) # complete the connection
workflow.connect(extracter, 'out_file', ds, 'extracted_timeseries')

# Make a graph to visualize what we did..
workflow.write_graph('graph.dot', format='png')
from IPython.display import Image
Image('/data/workflow_folders/extract_striatum_signal/graph.png')

And we're ready to run

In [ ]:
workflow.run('MultiProc', plugin_args={'n_proc': 3})

5) Again, visualize what the output data structure looks like using `!tree`

In [ ]:
# your code here